In [6]:
from __future__ import annotations
import os, re
from pathlib import Path
from typing import Dict, Optional

import numpy as np
import pandas as pd
import warnings

In [7]:
#Utils

def ensure_dirs(paths):
    """
    Garante que todas as pastas em 'paths' existem.
    paths pode ser lista de strings ou uma única string.
    """
    if isinstance(paths, str):
        paths = [paths]
    for p in paths:
        os.makedirs(p, exist_ok=True)

def to_float_smart(x):
    """
    Converte strings tipo '1.234,56' (BR) ou '1,234.56' (US) e variantes em float.
    Trata negativos e milhares. Retorna NaN se não der.
    """
    import re
    if pd.isna(x):
        return np.nan
    if isinstance(x, (int, float, np.number)):
        return float(x)

    s = str(x).strip()
    if s in {"", "-", "--", "nan", "NaN", "None", "NULL"}:
        return np.nan

    # mantém apenas dígitos, sinais e separadores
    s = re.sub(r"[^0-9\-\.,]", "", s)

    has_dot   = "." in s
    has_comma = "," in s

    try:
        if has_dot and has_comma:
            # decide pelo separador mais à direita
            if s.rfind(",") > s.rfind("."):
                # BR: 1.234,56 -> 1234.56
                s = s.replace(".", "").replace(",", ".")
            else:
                # US: 1,234.56 -> 1234.56
                s = s.replace(",", "")
            return float(s)

        if has_comma and not has_dot:
            # BR decimal: 1234,56 -> 1234.56
            return float(s.replace(",", "."))

        if has_dot and not has_comma:
            # Pode ser decimal (um ponto) ou milhares (vários pontos)
            if s.count(".") == 1:
                return float(s)  # 1234.56
            else:
                # 109.641.290.194 -> 109641290194
                return float(s.replace(".", ""))

        # Só dígitos e talvez sinal
        return float(s)
    except Exception:
        return np.nan



def parse_date_br_any(sr: pd.Series) -> pd.Series:
    """
    Converte a coluna de datas de publicação para datetime,
    assumindo SEMPRE padrão brasileiro (DD/MM/AAAA) quando houver ambiguidade.

    Regras:
    - Se estiver no padrão ISO 'YYYY-MM-DD', usamos isso direto (não é ambíguo).
    - Se estiver no padrão brasileiro 'DD/MM/YYYY', interpretamos como dia/mês/ano.
    - Se vier em qualquer outro formato, tentamos parse com dayfirst=True.
    - No final, retornamos datetime normalizado (sem hora).
    """

    s = sr.astype(str).str.strip()

    # 1) tenta ISO claro: 2024-03-31
    iso_mask = s.str.match(r"^\d{4}-\d{2}-\d{2}$")
    out_iso = pd.to_datetime(
        s.where(iso_mask),
        format="%Y-%m-%d",
        errors="coerce"
    )

    # 2) tenta BR claro: 31/03/2024
    br_mask = s.str.match(r"^\d{2}/\d{2}/\d{4}$")
    out_br = pd.to_datetime(
        s.where(br_mask),
        format="%d/%m/%Y",
        dayfirst=True,
        errors="coerce"
    )

    # 3) começa com ISO e preenche lacunas com BR
    out = out_iso.fillna(out_br)

    # 4) fallback genérico:
    #    qualquer coisa que sobrou a gente interpreta assumindo padrão brasileiro (dayfirst=True)
    still_nat = out.isna()
    if still_nat.any():
        out_fallback = pd.to_datetime(
            s[still_nat],
            errors="coerce",
            dayfirst=True   # <- força semântica brasileira
        )
        out.loc[still_nat] = out_fallback

    # 5) normaliza para "apenas a data" (zera hora)
    out = out.dt.normalize()

    return out
        
# =========================================
# Core helpers: índice do evento, beta, CAR
# =========================================

   
def detect_start_index(prices: pd.DataFrame,
                       event_date: pd.Timestamp) -> int:
    """
    Retorna o índice T1 = primeiro pregão APÓS o pregão marcado como evento.

    event_date: data do pregão em que 'event == 1' (earnings divulgado).
    T1 = índice do próximo pregão (reação), não o próprio dia do evento.
    """

    if prices is None or prices.empty or "Date" not in prices:
        return 0

    pr_sorted = prices.sort_values("Date").reset_index(drop=True)

    # achar o índice da data do evento em si
    hit = pr_sorted.index[pr_sorted["Date"] == event_date]

    if len(hit) > 0:
        base_idx = int(hit[0])
        # queremos o pregão seguinte
        t1_idx = base_idx + 1
        # clamp para não sair do range
        if t1_idx >= len(pr_sorted):
            t1_idx = len(pr_sorted) - 1
        return t1_idx

    # fallback: se a data exata não está no df (não deveria acontecer se veio de event==1),
    # achar posição >= event_date e pular pro seguinte
    ds = pr_sorted["Date"].values
    idx_ge = np.searchsorted(ds, np.array(event_date, dtype="datetime64[ns]"))
    t1_idx = idx_ge + 1
    if t1_idx >= len(pr_sorted):
        t1_idx = len(pr_sorted) - 1
    return int(t1_idx)
 



def estimate_beta(stock_df: pd.DataFrame,
                  mkt_df: pd.DataFrame,
                  rf_df: pd.DataFrame,
                  event_idx: int,
                  estimation_window: int = 504) -> float:

    stock_df = stock_df.sort_values('Date').reset_index(drop=True)
    mkt_df   = mkt_df.sort_values('Date').reset_index(drop=True)
    rf_df    = rf_df.sort_values('Date').reset_index(drop=True)

    m = stock_df[['Date','Close']].merge(
            mkt_df[['Date','Close']], on='Date', suffixes=('_i','_m')
        )
    m = m.merge(
            rf_df[['Date','rf_daily']],
            on='Date',
            how='left'
        ).ffill().sort_values('Date').reset_index(drop=True)

    m["Close_i"]  = m["Close_i"].apply(to_float_smart).astype(float)
    m["Close_m"]  = m["Close_m"].apply(to_float_smart).astype(float)
    m["rf_daily"] = m["rf_daily"].apply(to_float_smart).astype(float)

    m['ri'] = np.log(m['Close_i'] / m['Close_i'].shift(1))
    m['rm'] = np.log(m['Close_m'] / m['Close_m'].shift(1))

    if event_idx < 2:
        return np.nan
    if event_idx >= len(stock_df):
        return np.nan

    event_date = stock_df.iloc[event_idx]['Date']

    eidx_arr = m.index[m['Date'] == event_date]
    if len(eidx_arr) == 0:
        return np.nan
    eidx = int(eidx_arr[0])

    start = max(m.index.min(), eidx - estimation_window)
    end   = eidx - 1

    # precisa de histórico mínimo pra regressão
    if end - start < 30:
        return np.nan

    win = m.loc[start:end].dropna()
    if win.empty:
        return np.nan

    x = (win['rm'] - win['rf_daily']).values.reshape(-1, 1)
    y = (win['ri'] - win['rf_daily']).values.reshape(-1, 1)

    if x.shape[0] < 5:
        return np.nan

    beta = np.linalg.lstsq(x, y, rcond=None)[0].ravel()[0]
    return float(beta)


def compute_car(stock_df: pd.DataFrame,
                mkt_df: pd.DataFrame,
                rf_df: pd.DataFrame,
                event_idx: int,
                beta: float,
                holding_days: int = 30) -> float:

    stock_df = stock_df.sort_values('Date').reset_index(drop=True)
    mkt_df   = mkt_df.sort_values('Date').reset_index(drop=True)
    rf_df    = rf_df.sort_values('Date').reset_index(drop=True)

    m = stock_df[['Date','Close']].merge(
            mkt_df[['Date','Close']], on='Date', suffixes=('_i','_m')
        )
    m = m.merge(
            rf_df[['Date','rf_daily']],
            on='Date',
            how='left'
        ).ffill().sort_values('Date').reset_index(drop=True)

    m["Close_i"]  = m["Close_i"].apply(to_float_smart).astype(float)
    m["Close_m"]  = m["Close_m"].apply(to_float_smart).astype(float)
    m["rf_daily"] = m["rf_daily"].apply(to_float_smart).astype(float)

    m['ri'] = np.log(m['Close_i'] / m['Close_i'].shift(1))
    m['rm'] = np.log(m['Close_m'] / m['Close_m'].shift(1))

    start = event_idx
    end   = min(start + holding_days - 1, len(m) - 1)

    seg = m.iloc[start:end+1].dropna()
    if seg.empty:
        return np.nan

    seg['E_ri'] = seg['rf_daily'] + beta * (seg['rm'] - seg['rf_daily'])
    seg['AR']   = seg['ri'] - seg['E_ri']

    return float(seg['AR'].sum())

In [8]:
# Carregadores de mercado
# =========================================
class MarketAndRiskLoader:
    @staticmethod
    def load_ibov_csv(path_ibov: str) -> pd.DataFrame:
        df = pd.read_csv(path_ibov, dtype=str)
        df["Data"] = pd.to_datetime(df["Data"], dayfirst=True, errors="coerce")

        for c in ["FechAjust","FechHist"]:
            if c in df.columns:
                df[c] = df[c].apply(to_float_smart)

        close_col = "FechAjust" if "FechAjust" in df.columns and df["FechAjust"].notna().any() else "FechHist"

        out = (df[["Data", close_col]]
               .rename(columns={"Data":"Date", close_col:"Close"})
               .dropna(subset=["Date","Close"])
               .sort_values("Date")
               .reset_index(drop=True))

        out["Close"] = out["Close"].astype(float)
        return out

    @staticmethod
    def load_cdi_csv(path_cdi: str) -> pd.DataFrame:
        df = pd.read_csv(path_cdi, dtype=str)
        df["Data"] = pd.to_datetime(df["Data"], dayfirst=True, errors="coerce")

        if "Var" in df.columns:
            df["Var"] = df["Var"].apply(to_float_smart)
            if df["Var"].notna().sum() > 3:
                out = df[["Data"]].copy()
                out["rf_daily"] = df["Var"] / 100.0
                out = (out.rename(columns={"Data":"Date"})
                          .dropna()
                          .sort_values("Date")
                          .reset_index(drop=True))
                out["rf_daily"] = out["rf_daily"].astype(float)
                return out

        for c in ["FechAjust","FechHist"]:
            if c in df.columns:
                df[c] = df[c].apply(to_float_smart)
                if df[c].notna().sum() > 3:
                    r = df[c].pct_change()
                    out = df[["Data"]].copy()
                    out["rf_daily"] = r
                    out = (out.rename(columns={"Data":"Date"})
                              .dropna()
                              .sort_values("Date")
                              .reset_index(drop=True))
                    out["rf_daily"] = out["rf_daily"].astype(float)
                    return out

        return pd.DataFrame(columns=["Date","rf_daily"])

In [9]:
# Builder de eventos PEAD / CAR
# =========================================
class EventDatasetBuilder:
    def __init__(self, 
                 mkt_df: pd.DataFrame, 
                 rf_df: pd.DataFrame,
                 estimation_window: int = 504, 
                 holding_days: int = 30, 
                 min_estimation: int = 60):
        self.mkt = mkt_df.sort_values('Date').reset_index(drop=True)
        self.rf  = rf_df.sort_values('Date').reset_index(drop=True)
        self.estimation_window = estimation_window
        self.holding_days      = holding_days
        self.min_estimation    = min_estimation

    def build_for_ticker(self, 
                         tkr: str,
                         price_csv: str,
                         fund_csv: str) -> pd.DataFrame:
        # ====== 1. Carrega preços processados ======
        px = pd.read_csv(price_csv, dtype=str)

#        # Data em datetime BR
        px["Data"] = parse_date_br_any(px["Data"])

#        # Close em float
#        if "Close" in px.columns:
#            px["Close"] = px["Close"].apply(to_float_smart)
#        elif "FechAjust" in px.columns:
#            px["Close"] = px["FechAjust"].apply(to_float_smart)
#        elif "FechHist" in px.columns:
#            px["Close"] = px["FechHist"].apply(to_float_smart)
#        else:
#            return pd.DataFrame()

        # event como inteiro (0/1)
        if "event" not in px.columns:
            return pd.DataFrame()
        px["event"] = pd.to_numeric(px["event"], errors="coerce").fillna(0).astype(int)

        # limpa linhas inválidas
        px = (px.dropna(subset=["Data","Close"])
                .sort_values("Data")
                .reset_index(drop=True))
        #px["Close"] = px["Close"].astype(float)

        # dataframes auxiliares usados em beta e CAR
        px_idx   = px[["Data"]].rename(columns={"Data":"Date"})         # só datas
        px_close = px[["Data","Close"]].rename(columns={"Data":"Date"}) # datas + preço

        # ====== 2. Definir quais são os eventos ======
        # AnnounceDate = cada pregão onde event == 1
        event_rows = px[px["event"] == 1].copy().sort_values("Data")
        announce_dates = event_rows["Data"].tolist()  # list de Timestamp

        if len(announce_dates) == 0:
            return pd.DataFrame()

        # ====== 3. Carrega fundamentals só para anexar features ======
        fund = pd.read_csv(fund_csv, dtype=str)

        if "AnnounceDate" in fund.columns:
            fund["AnnounceDate"] = parse_date_br_any(fund["AnnounceDate"])
        elif "Data_Publicacao" in fund.columns:
            fund["AnnounceDate"] = parse_date_br_any(fund["Data_Publicacao"])
        else:
            fund["AnnounceDate"] = pd.NaT

        fund = (fund.dropna(subset=["AnnounceDate"])
                    .sort_values("AnnounceDate")
                    .reset_index(drop=True))
        fund_feats = fund.copy()

        # ====== 4. Loop de cada evento ======
        recs = []

        for announced_at in announce_dates:
            # announced_at: Timestamp do pregão onde event == 1
            # t1_idx: índice do PRIMEIRO pregão DEPOIS do evento
            t1_idx = detect_start_index(px_idx, announced_at)

            # histórico suficiente p/ estimar beta?
            est_len = min(self.estimation_window, t1_idx)
            if est_len < self.min_estimation:
                continue

            beta = estimate_beta(px_close, self.mkt, self.rf, t1_idx, est_len)
            if pd.isna(beta):
                continue

            car  = compute_car(px_close, self.mkt, self.rf, t1_idx, beta, self.holding_days)
            if pd.isna(car):
                continue
            
            

            trade_date = px_idx.iloc[t1_idx]["Date"]  # pregão usado como início do CAR (T+1)

            row = {
                "Ticker"        : tkr,
                # AnnounceDate agora é o pregão marcado com event == 1 (T0)
                "AnnounceDate"  : announced_at.strftime("%d/%m/%Y") if not pd.isna(announced_at) else "",
                # EventTradeDate é T+1 = início da janela CAR
                "EventTradeDate": trade_date.strftime("%d/%m/%Y") if not pd.isna(trade_date) else "",
                "CAR_30D"       : float(car),
                "CAR_Sign"      : int(car > 0),
                "Beta"          : float(beta),
                "EstimationLen" : int(est_len),
                "FundSource"    : os.path.basename(fund_csv),
            }

            # opcional: anexar features fundamentalistas "válidas até aquele evento"
            if not fund_feats.empty:
                # pega a última linha de fund com AnnounceDate <= announced_at
                mask = fund_feats["AnnounceDate"] <= announced_at
                if mask.any():
                    fmatch = fund_feats.loc[mask].iloc[-1]
                    extras = fmatch.drop(labels=["AnnounceDate"], errors="ignore").to_dict()
                    row.update(extras)

            recs.append(row)

        out_df = pd.DataFrame(recs)

        if "AnnounceDate" in out_df.columns:
            # como agora AnnounceDate é string BR "DD/MM/AAAA",
            # vamos ordenar por data convertendo temporariamente
            order_helper = parse_date_br_any(out_df["AnnounceDate"])
            out_df = out_df.assign(_ord=order_helper).sort_values(["_ord","Ticker"]).drop(columns="_ord").reset_index(drop=True)

        return out_df 


In [5]:
# Execução principal (gera arquivos finais)
# =========================================

if __name__ == "__main__":
    warnings.filterwarnings("ignore", category=FutureWarning)

    # 0) Pastas base
    ensure_dirs([
        "dataset/final",
        "dataset/prices_processed",
        "dataset/fund_processed",
        "dataset/prices",
    ])

    # 1) Carrega IBOV e CDI
    mkt = MarketAndRiskLoader.load_ibov_csv("dataset/prices/IBOV.SA.csv")  # -> Date, Close
    rf  = MarketAndRiskLoader.load_cdi_csv("dataset/prices/CDI.SA.csv")    # -> Date, rf_daily

    # 2) Builder de eventos CAR/Beta
    builder = EventDatasetBuilder(
        mkt_df=mkt,
        rf_df=rf,
        estimation_window=504,  # ~2 anos úteis
        holding_days=30,        # CAR de 30 dias úteis
        min_estimation=60       # exige pelo menos ~3 meses de histórico antes do evento
    )

    all_events = []

    # 3) Loop pelos tickers já processados
    prices_dir = Path("dataset/prices_processed")
    fund_dir   = Path("dataset/fund_processed")
    final_dir  = Path("dataset/final")

    for price_file in prices_dir.glob("*SA.csv"):
        # Inferir ticker base: "PETR4.SA.csv" -> "PETR4"
        base_name = price_file.stem      # "PETR4.SA"
        tkr = base_name.split(".")[0]    # "PETR4"

        fund_file = fund_dir / f"{tkr}.SA.csv"
        if not fund_file.exists():
            # se não tem fundamentals pra esse ticker, pula
            continue

        ev_df = builder.build_for_ticker(
            tkr=tkr,
            price_csv=str(price_file),
            fund_csv=str(fund_file),
           
        )

        if ev_df is None or ev_df.empty:
            continue

        # salva arquivo final por ticker:
        # dataset/final/{tkr}_pead_event.csv
        out_tkr_path = final_dir / f"{tkr}_pead_event.csv"
        ev_df.to_csv(out_tkr_path, index=False)

        all_events.append(ev_df)

    # 4) Consolidado geral
    if all_events:
        events_df = pd.concat(all_events, ignore_index=True)

        # Ordenar também o consolidado por AnnounceDate
        if "AnnounceDate" in events_df.columns:
            events_df = (
                events_df.sort_values(["AnnounceDate","Ticker"])
                         .reset_index(drop=True)
            )

        # salva dataset consolidado
        out_all_path = final_dir / "pead_event_dataset_2010_2019.csv"
        events_df.to_csv(out_all_path, index=False)

        print("OK — dataset consolidado salvo em dataset/final/pead_event_dataset_2010_2019.csv")
        print(events_df.head())
    else:
        print("Nenhum evento encontrado. Verifique se 'AnnounceDate' está presente em fund_processed/*.csv e se há preços válidos em prices_processed/*.csv.")

OK — dataset consolidado salvo em dataset/final/pead_event_dataset_2010_2019.csv
   Ticker AnnounceDate EventTradeDate   CAR_30D  CAR_Sign      Beta  \
0   FIBR3   01/02/2012     02/02/2012 -0.009841         0  1.255711   
1   CIEL3   01/02/2016     02/02/2016 -0.095911         0  0.636058   
2  KLBN11   01/02/2018     02/02/2018  0.071078         1  0.332900   
3   KLBN4   01/02/2018     02/02/2018  0.008948         1  0.360474   
4   MDIA3   01/03/2011     02/03/2011  0.112207         1  0.390996   

   EstimationLen     FundSource        Data       Empresa  ...  \
0            504   FIBR3.SA.csv  31/12/2011        FIBRIA  ...   
1            504   CIEL3.SA.csv  31/12/2015         CIELO  ...   
2            504  KLBN11.SA.csv  31/12/2017    KLABIN S/A  ...   
3            504   KLBN4.SA.csv  31/12/2017    KLABIN S/A  ...   
4            287   MDIA3.SA.csv  30/09/2010  M.DIASBRANCO  ...   

          PNC_Y_Change Outros_PC_MET   Outros_PC_Q_Change  \
0  -178.14300000000003       272.4